In [2]:
from pathlib import Path
from napari.qt import thread_worker
import napari
from napari.types import LabelsData, LayerDataTuple
from napari.layers import Image
from napari.layers import Labels
from qtpy.QtCore import Qt
from superqt import QLabeledDoubleRangeSlider

import scanreader

In [3]:
parent = Path('/home/rbo/caiman_data')
raw_tiff_name = parent / 'high_res.tif'
reader = scanreader.read_scan(str(raw_tiff_name), join_contiguous=True)

In [7]:
def trim(scan, amounts_x):
    new_slice_x = [slice(s.start + amounts_x[0], s.stop - amounts_x[1]) for s in scan.fields[0].output_xslices]
    return [i for s in new_slice_x for i in range(s.start, s.stop)]

def get_slice(reader,y,x,z,t):
    return reader[:,y,x,z,t]

def update_image(viewer, tleft=0, tright=0):
    print('Calling update image')
    trim_x = trim(reader, (tleft,tright))
    arr = reader[:, :, trim_x, 0, 2]
    viewer.layers[0].data = arr
    viewer.update()

In [12]:
from magicgui import magicgui
viewer = napari.Viewer()
layer = viewer.add_image(reader[:,:,:,0,:].squeeze().transpose(2,0,1), name='image')

/home/rbo/miniconda3/envs/mbo-mesmerize/lib/python3.10/site-packages/napari/plugins/_plugin_manager.py:560: UserWarning: Plugin 'napari-animated-gif-io' has already registered a function widget 'save as animated gif' which has now been overwritten
  warn(message=warn_message)


In [1]:


@layer.mouse_drag_callbacks.append
def update_layer(layer, event):
    layer.data = reader[]

def update_image(amounts_x_start: int = 0, amounts_x_end: int = 0):
    tx = trim(reader, (amounts_x_start, amounts_x_end))
    trimmed = reader[:, :, tx, 0, 2]
    
    # Update the viewer with the new image data
    if 'image' in viewer.layers:
        viewer.layers['image'].data = trimmed
    else:
        viewer.add_image(trimmed, name='image', colormap='gray')

# Connect the sliders to the update function directly
update_image.amounts_x_start.changed.connect(lambda event: update_image())
update_image.amounts_x_end.changed.connect(lambda event: update_image())

viewer=napari.Viewer()
# Add the widget to the viewer
viewer.window.add_dock_widget(update_image, area='bottom')


NameError: name 'napari' is not defined

In [20]:
if not viewer:
    viewer = napari.viewer()
else:
    viewer = napari.current_viewer()
    viewer.add_image(reader[:,:,:,0,2].squeeze())
viewer.window.add_dock_widget(my_slider, area='bottom')

In [22]:
import typing
from typing import Annotated
from magicgui import magicgui, widgets

w1 = widgets.Slider(value=15, max=25, label='Slider:')
w2 = widgets.FloatSlider(value=10.5, max=18.5, label='FloatSlider:')
w3 = widgets.ProgressBar(value=80, max=100, label='ProgressBar:')
container = widgets.Container(widgets=[w1, w2, w3])

def re_trim(
    layer: 'napari.layers.Image',
    xtrim_left: Annotated[float, {'widget_type': 'SpinBox', 'min':0, 'max': 10}] = 0,
    xtrim_right: Annotated[float, {'widget_type': 'SpinBox', 'min':0, 'max': 10}] = 0,
) -> 'typing.Optional[napari.types.ImageData]':
    if layer:
        new_trim = trim(reader, (xtrim_left, xtrim_right))
        return reader[:,:,new_trim,0,2]
    return None

# create a viewer and add some images
viewer = napari.Viewer()
viewer.add_image(reader[:,:,:,0,2].squeeze(), name='astronaut')

viewer.window.add_function_widget(re_trim)

/home/rbo/miniconda3/envs/mbo-mesmerize/lib/python3.10/site-packages/napari/plugins/_plugin_manager.py:560: UserWarning: Plugin 'napari-animated-gif-io' has already registered a function widget 'save as animated gif' which has now been overwritten
  warn(message=warn_message)


In [ ]:

w1 = widgets.SpinBox(value=1, max=reader.num_channels, label='z-Plane:')